<a href="https://colab.research.google.com/github/muesgit/deep_learning_tutorial/blob/main/FishVsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Mount Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Build Training Dataset
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

train_data_path = '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/train'
# to secure the model from overfitting, the validation data set is created
# after each training cycle (epoch) it is compared against it
val_data_path = '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/val'
# test model after all training has been completed
test_data_path = '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/test'

# Check if image from the dataset can be opened, otherwise it wont be used
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

# Transformation routine
transforms = transforms.Compose([
    # Scale incoming image to defined Resolution
    transforms.Resize((64, 64)),
    # Convert image to tensor
    transforms.ToTensor(),
    # normalize image to keep the incoming values between 0 and 1
    # (exploding gradient problem), Values from ImageNet dataset
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data  = torchvision.datasets.ImageFolder(root=train_data_path,
                                              transform=transforms, is_valid_file=check_image)
val_data    = torchvision.datasets.ImageFolder(root=val_data_path,
                                            transform=transforms, is_valid_file=check_image)
test_data   = torchvision.datasets.ImageFolder(root=test_data_path,
                                             transform=transforms, is_valid_file=check_image)

Dataset ImageFolder
    Number of datapoints: 801
    Root location: /content/drive/MyDrive/Colab Notebooks/Pytorch_Book/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [20]:
import torch.utils.data
from torch.utils.data import DataLoader
# Build Loader
# Data Loader is pushing data into the neural network for training
# Batch Size (mini-batches require less memory)
batch_size = 64
train_data_loader = DataLoader(train_data, batch_size=batch_size)
val_data_loader   = DataLoader(val_data, batch_size=batch_size)
test_data_loader  = DataLoader(test_data, batch_size=batch_size)

In [21]:
# Building fully connected neural network
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # the fully connected layers, numbers in between are arbitraty
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        # last layer has size of classes (two: Cat, Fish)
        self.fc3 = nn.Linear(50, 2)
    def forward(self, x):
        # transform x (datashape)
        x = x.view(-1, 12288)
        # relu, rectified linear output
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # since CrossEntropyLoss function is used for Loss determination,
        # no softmax is used for the last layer
        x = self.fc3(x)
        return x

simplenet = SimpleNet()

In [22]:
# Create an optimizer
import torch.optim as optim

# lr := Learning Rate (increment)
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs, device):
  for epoch in range(epochs):

    training_loss = 0.0
    valid_loss = 0.0
    model.train()

    for batch in train_loader:
      # Resetting Gradients, only use gradients of current branch
      optimizer.zero_grad()
      inputs, labels = batch
      # move data to gpu
      inputs = inputs.to(device)
      labels = labels.to(device)
      output = model(inputs)
      # Calculate the loss (difference between test and prediction)
      loss = loss_fn(output, labels)
      # .backward:= compute gradients
      loss.backward()
      # .step:= adjust the weights
      optimizer.step()
      training_loss += loss.data.item()
    training_loss /= len(train_loader)

    model.eval()
    num_correct = 0
    num_examples = 0

    for batch in val_loader:
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      output = model(inputs)
      loss = loss_fn(output, labels)
      valid_loss += loss.data.item()
      correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], labels).view(-1)
      num_correct += torch.sum(correct).item()
      num_examples += correct.shape[0]
    valid_loss /= len(val_loader)

    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.
          format(epoch, training_loss, valid_loss, num_correct / num_examples))

In [23]:
# Load data into gpu if is available
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

epochs = 20
simplenet.to(device)
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs, device)

Epoch: 0, Training Loss: 2.97, Validation Loss: 6.72, accuracy = 0.40
Epoch: 1, Training Loss: 3.39, Validation Loss: 1.77, accuracy = 0.60
Epoch: 2, Training Loss: 1.29, Validation Loss: 0.81, accuracy = 0.57
Epoch: 3, Training Loss: 0.73, Validation Loss: 0.74, accuracy = 0.73
Epoch: 4, Training Loss: 0.46, Validation Loss: 0.79, accuracy = 0.70
Epoch: 5, Training Loss: 0.40, Validation Loss: 0.73, accuracy = 0.70
Epoch: 6, Training Loss: 0.32, Validation Loss: 0.76, accuracy = 0.71
Epoch: 7, Training Loss: 0.27, Validation Loss: 0.78, accuracy = 0.68
Epoch: 8, Training Loss: 0.24, Validation Loss: 0.74, accuracy = 0.71
Epoch: 9, Training Loss: 0.20, Validation Loss: 0.79, accuracy = 0.70
Epoch: 10, Training Loss: 0.19, Validation Loss: 0.77, accuracy = 0.68
Epoch: 11, Training Loss: 0.17, Validation Loss: 0.83, accuracy = 0.67
Epoch: 12, Training Loss: 0.16, Validation Loss: 0.79, accuracy = 0.69
Epoch: 13, Training Loss: 0.13, Validation Loss: 0.79, accuracy = 0.72
Epoch: 14, Train

In [26]:
# Make predictions
labels = ['Cat', 'Fish']

# example image
img = Image.open('/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/test/cat/2071455892_689f59db15.jpg')
img = transforms(img).to(device)

# use model to make prediction
prediction = simplenet(img)
# get index of highest value
prediction = prediction.argmax()
print(labels[prediction])

Cat


In [28]:
# Save model
# save structure an parameter
torch.save(simplenet, '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/model/simplenet')
# only save models state
torch.save(simplenet.state_dict(), '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/model/simplenet_state')